# Architecture

# Folder Structure

In [ ]:
+---docker
|       Dockerfile
|       pipeline_base_image_builder.sh
|       secrets.json
|
+---img
|       screenshots.png
|
+---train_pipeline
|       pipeline_components.py
|       pipeline_publisher.py
|       pipeline_run.py
|       cloudbuild.yaml
|
+---deployment
|       score.py
|       GKE_config.yaml
|       cloudbuild.yaml
|
+---tests
|       test_1_FE.py
|       test_2_impute_class.py
|       test_3_score.py
|
+---utils
|       FE.py
|       impute_class.py
|
|---architecture.pptx
|---main.ipynb
|---requirements.txt